In [1]:
import pandas as pd
from math import comb
from ax.service.ax_client import AxClient
import time
import sys

sys.path.append('../../../0_helper_functions')
import sdlnano as sdl
import calculation as calc

In [2]:
# Design space size calculation


def calculate_combinations(num_variables, total_sum, step):
    steps = int(total_sum / step)
    return comb(steps + num_variables - 1, num_variables - 1)

# Define parameters
step = 0.05
total_sum = 1

# Calculate combinations for each group
count_group1 = calculate_combinations(10, total_sum, step)  # 3 Solid lipids + 3 Polymers + 3 Liquid lipids + 1 Drug
count_group2 = calculate_combinations(4, total_sum, step)   # 3 Surfactants + 1 Water

# Calculate total combinations
total_combinations = count_group1 * count_group2


print("organic design space size: ", round(count_group1/(1e6),2), 'Million')
print("aqueous design space size: ", round(count_group2/(1e3),2), 'Thousand')
print("design space size: ", round(total_combinations/(1e9),2), 'Billion')


organic design space size:  10.02 Million
aqueous design space size:  1.77 Thousand
design space size:  17.74 Billion


# Check the iteration number

In [3]:
iteration = sdl.get_iteration_number()
print("This is the iteration: #", iteration)

This is the iteration: # 7


# Check the drug

In [4]:
drug = 'ACE'
print("This is the drug:", drug)

This is the drug: ACE


# Optimizer initialization (N/A)

# Generate recommendations

In [5]:
time_start = time.time()

previous_optimizer_file_name = '../iteration_' + str(iteration-1) + '/optimizer/optimizer_load_' + str(iteration-1) + '.json'
# generate recomms
ax_client_init = AxClient.load_from_json_file(previous_optimizer_file_name)
unlabeled_trial, ax_client_design = sdl.generate_trials(ax_client=ax_client_init, num_of_trials = 16, drug = drug, bopt=1)

time_end = time.time()

print(round((time_end-time_start)/60), 'min')

/Users/zeqing/opt/anaconda3/envs/sdlnano_plot/lib/python3.11/site-packages/botorch/acquisition/multi_objective/logei.py:418: RuntimeWarning: `cache_root` is only supported for GPyTorchModels that are not MultiTask models and don't produce a TransformedPosterior. Got a model of type <class 'botorch.models.model.ModelList'>. Setting `cache_root = False`.
  NoisyExpectedHypervolumeMixin.__init__(
[INFO 08-07 06:36:35] ax.service.ax_client: Generated new trial 112 with parameters {'Drug': 98, 'SL_1': 73, 'SL_2': 0, 'SL_3': 0, 'LL_1': 0, 'LL_2': 0, 'LL_3': 0, 'P_1': 0, 'P_2': 0, 'P_3': 0, 'S_1': 1, 'S_2': 0, 'S_3': 0, 'Water': 0} using model BoTorch.
/Users/zeqing/opt/anaconda3/envs/sdlnano_plot/lib/python3.11/site-packages/ax/core/data.py:293: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the re

8 min


In [6]:
unlabeled_trial[['S_1', 'S_2', 'S_3', 'Water']]

,S_1,S_2,S_3,Water
0,1,0,0,0
1,1,0,1,0
2,0,0,0,0
3,0,0,1,0
4,0,0,0,0
5,0,0,0,0
6,0,0,0,0
7,0,0,1,0
8,0,0,0,0
9,0,0,0,0


In [7]:
mask = (unlabeled_trial[['S_1', 'S_2', 'S_3', 'Water']] == 0).all(axis=1)

# print the rows that are about to change
print("Rows to be updated (before):")
print(unlabeled_trial.loc[mask])

# set Water to 100 on those rows
unlabeled_trial.loc[mask, 'Water'] = 100

# print the rows after the update
print("\nRows after update:")
print(unlabeled_trial.loc[mask])

Rows to be updated (before):
    trial_index  Drug  SL_1  SL_2  SL_3  LL_1  LL_2  LL_3  P_1  P_2  ...  S_1  \
2           114    99    67     0     0     0     0     0    0    0  ...    0   
4           116    99    67     0     0     0     0     0    0    0  ...    0   
5           117    99    67     0     0     0     0     0    0    0  ...    0   
6           118    99    67     0     0     0     0     0    0    0  ...    0   
8           120    99    67     0     0     0     0     0    0    0  ...    0   
9           121    99    67     0     0     0     0     0    0    0  ...    0   
10          122    99    67     0     0     0     0     0    0    0  ...    0   
12          124    99    67     0     0     0     0     0    0    0  ...    0   
13          125    99    67     0     0     0     0     0    0    0  ...    0   

    S_2  S_3  Water  Solu Solu_STD  Size Size_STD   PDI PDI_STD  
2     0    0      0  None     None  None     None  None    None  
4     0    0      0  None   

In [8]:
unlabeled_trial[['S_1', 'S_2', 'S_3', 'Water']]

,S_1,S_2,S_3,Water
0,1,0,0,0
1,1,0,1,0
2,0,0,0,100
3,0,0,1,0
4,0,0,0,100
5,0,0,0,100
6,0,0,0,100
7,0,0,1,0
8,0,0,0,100
9,0,0,0,100


In [9]:
# process the trails into ratios

unlabeled_trial_processed = sdl.process_trails(unlabeled_trial)
unlabeled_trial_processed.describe()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water
count,16.000000,16.000000,16.000000,16.0,16.0,16.0,16.0,16.0000,16.0,16.0,16.0,16.00000,16.0,16.000000,16.000000
mean,119.500000,0.578625,0.419875,0.0,0.0,0.0,0.0,0.0015,0.0,0.0,0.0,0.15625,0.0,0.281250,0.562500
std,4.760952,0.021071,0.019297,0.0,0.0,0.0,0.0,0.0060,0.0,0.0,0.0,0.30104,0.0,0.406971,0.512348
min,112.000000,0.545000,0.404000,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.00000,0.0,0.000000,0.000000
25%,115.750000,0.557500,0.404000,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.00000,0.0,0.000000,0.000000
50%,119.500000,0.596000,0.404000,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.00000,0.0,0.000000,1.000000
75%,123.250000,0.596000,0.442000,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.12500,0.0,0.500000,1.000000
max,127.000000,0.596000,0.448000,0.0,0.0,0.0,0.0,0.0240,0.0,0.0,0.0,1.00000,0.0,1.000000,1.000000


In [10]:
unlabeled_trial
unlabeled_trial.describe()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water
count,16.000000,16.000000,16.000000,16.0,16.0,16.0,16.0,16.00,16.0,16.0,16.0,16.000000,16.0,16.000,16.000000
mean,119.500000,95.750000,69.375000,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.250000,0.0,0.375,56.250000
std,4.760952,4.219005,2.801785,0.0,0.0,0.0,0.0,1.00,0.0,0.0,0.0,0.447214,0.0,0.500,51.234754
min,112.000000,90.000000,67.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.000,0.000000
25%,115.750000,91.000000,67.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.000,0.000000
50%,119.500000,99.000000,67.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.000,100.000000
75%,123.250000,99.000000,72.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.250000,0.0,1.000,100.000000
max,127.000000,99.000000,73.000000,0.0,0.0,0.0,0.0,4.00,0.0,0.0,0.0,1.000000,0.0,1.000,100.000000


In [11]:
# Convert ratios to volumes

transfer = calc.converter(unlabeled_trial_processed)
transfer.describe()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water,Solvent
count,16.000000,16.000000,16.000000,16.0,16.0,16.0,16.0,16.000,16.0,16.0,16.0,16.000000,16.0,16.000000,16.000000,16.0
mean,119.500000,202.518750,146.956250,0.0,0.0,0.0,0.0,0.525,0.0,0.0,0.0,156.250000,0.0,281.250000,562.500000,0.0
std,4.760952,7.374819,6.754033,0.0,0.0,0.0,0.0,2.100,0.0,0.0,0.0,301.039864,0.0,406.970515,512.347538,0.0
min,112.000000,190.750000,141.400000,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0
25%,115.750000,195.125000,141.400000,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0
50%,119.500000,208.600000,141.400000,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.000000,0.0,0.000000,1000.000000,0.0
75%,123.250000,208.600000,154.700000,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,125.000000,0.0,500.000000,1000.000000,0.0
max,127.000000,208.600000,156.800000,0.0,0.0,0.0,0.0,8.400,0.0,0.0,0.0,1000.000000,0.0,1000.000000,1000.000000,0.0


In [12]:
#dead_volume = 3.0 # mL

round(transfer[['Drug','SL_1', 'SL_2', 'SL_3', 'LL_1', 'LL_2', 'LL_3', 'P_1', 'P_2', 'P_3', 'S_1', 'S_2', 'S_3', 'Water', 'Solvent']].sum()/1000,1)

Drug       3.2
SL_1       2.4
SL_2       0.0
SL_3       0.0
LL_1       0.0
LL_2       0.0
LL_3       0.0
P_1        0.0
P_2        0.0
P_3        0.0
S_1        2.5
S_2        0.0
S_3        4.5
Water      9.0
Solvent    0.0
dtype: float64

In [13]:
# file path
ax_client_design_path = "optimizer/optimizer_design_" + str(iteration) + ".json"

unlabeled_trial_path = "data/unlabeled_" + str(iteration) + ".xlsx"

unlabeled_trial_processed_path = "data/unlabeled_processed_" + str(iteration) + ".xlsx"

transfer_path = 'data/transfer_' + str(iteration) + '.xlsx'


In [14]:
ax_client_design.save_to_json_file(ax_client_design_path)

unlabeled_trial.to_excel(unlabeled_trial_path, index=False)

unlabeled_trial_processed.to_excel(unlabeled_trial_processed_path, index=False)

transfer.to_excel(transfer_path, index = False)

# Generate new protocol

In [15]:
# Paths
template_protocol = '../../templates/OT_protocol_template.py'

output_script_path_1 = 'protocol/iteration_' + str(iteration) + '_OT_2_protocol_1.py'
output_script_path_2 = 'protocol/iteration_' + str(iteration) + '_OT_2_protocol_2.py'



In [16]:
# Update the script with Excel data
updated_script = calc.update_transfer_script(template_protocol, transfer_path, output_script_path_1, output_script_path_2) 


In [17]:
updated_script

('protocol/iteration_7_OT_2_protocol_1.py',
 'protocol/iteration_7_OT_2_protocol_2.py')

# Perform experiment

# Results

In [18]:
results = calc.parameters_and_complexity(iteration)
results

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water,Complexity,Complexity_STD
0,112,0.573,0.427,0,0,0,0,0.000,0,0,0,1.0,0,0.0,0,2,0
1,113,0.558,0.442,0,0,0,0,0.000,0,0,0,0.5,0,0.5,0,3,0
2,114,0.596,0.404,0,0,0,0,0.000,0,0,0,0.0,0,0.0,1,1,0
3,115,0.556,0.444,0,0,0,0,0.000,0,0,0,0.0,0,1.0,0,2,0
4,116,0.596,0.404,0,0,0,0,0.000,0,0,0,0.0,0,0.0,1,1,0
5,117,0.596,0.404,0,0,0,0,0.000,0,0,0,0.0,0,0.0,1,1,0
6,118,0.596,0.404,0,0,0,0,0.000,0,0,0,0.0,0,0.0,1,1,0
7,119,0.552,0.448,0,0,0,0,0.000,0,0,0,0.0,0,1.0,0,2,0
8,120,0.596,0.404,0,0,0,0,0.000,0,0,0,0.0,0,0.0,1,1,0
9,121,0.596,0.404,0,0,0,0,0.000,0,0,0,0.0,0,0.0,1,1,0


In [19]:
size_raw = calc.size_raw(iteration)
size_raw

,Data Quality,Item,Size,PD Index
0,Good,A1,261.480349,0.208116
1,Good,A2,260.871584,0.214427
2,Good,A3,253.825015,0.145992
3,Good,A4,215.951675,0.161324
4,Good,A5,209.809023,0.207971
5,Good,A6,208.500612,0.182173
6,Caution,B1,238.278532,0.228790
7,Caution,B2,254.751452,0.213285
8,Caution,B3,232.367336,0.196795
9,Good,B4,232.575488,0.253216


In [20]:
size_processed = calc.process_formulations(size_raw)
size_processed

,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality
0,Formulation 1,258.725650,4.254975,0.189512,0.037821,1
1,Formulation 2,211.420437,3.978326,0.183822,0.023367,1
2,Formulation 3,1000.000000,0.000000,1.000000,0.000000,0
3,Formulation 4,1000.000000,0.000000,1.000000,0.000000,0
4,Formulation 5,1000.000000,0.000000,1.000000,0.000000,0
5,Formulation 6,1000.000000,0.000000,1.000000,0.000000,0
6,Formulation 7,224.427667,3.430782,0.217428,0.097031,1
7,Formulation 8,242.566169,11.567519,0.371350,0.000810,1
8,Formulation 9,1000.000000,0.000000,1.000000,0.000000,0
9,Formulation 10,1000.000000,0.000000,1.000000,0.000000,0


In [21]:
results_final = pd.concat([results, size_processed], axis=1)
results_final

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,...,S_3,Water,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality
0,112,0.573,0.427,0,0,0,0,0.000,0,0,...,0.0,0,2,0,Formulation 1,258.725650,4.254975,0.189512,0.037821,1
1,113,0.558,0.442,0,0,0,0,0.000,0,0,...,0.5,0,3,0,Formulation 2,211.420437,3.978326,0.183822,0.023367,1
2,114,0.596,0.404,0,0,0,0,0.000,0,0,...,0.0,1,1,0,Formulation 3,1000.000000,0.000000,1.000000,0.000000,0
3,115,0.556,0.444,0,0,0,0,0.000,0,0,...,1.0,0,2,0,Formulation 4,1000.000000,0.000000,1.000000,0.000000,0
4,116,0.596,0.404,0,0,0,0,0.000,0,0,...,0.0,1,1,0,Formulation 5,1000.000000,0.000000,1.000000,0.000000,0
5,117,0.596,0.404,0,0,0,0,0.000,0,0,...,0.0,1,1,0,Formulation 6,1000.000000,0.000000,1.000000,0.000000,0
6,118,0.596,0.404,0,0,0,0,0.000,0,0,...,0.0,1,1,0,Formulation 7,224.427667,3.430782,0.217428,0.097031,1
7,119,0.552,0.448,0,0,0,0,0.000,0,0,...,1.0,0,2,0,Formulation 8,242.566169,11.567519,0.371350,0.000810,1
8,120,0.596,0.404,0,0,0,0,0.000,0,0,...,0.0,1,1,0,Formulation 9,1000.000000,0.000000,1.000000,0.000000,0
9,121,0.596,0.404,0,0,0,0,0.000,0,0,...,0.0,1,1,0,Formulation 10,1000.000000,0.000000,1.000000,0.000000,0


In [22]:
# µg/mL
results_final['Solu'] = results_final['Drug'] * 600 / 0.3
results_final['Solu_STD'] = 0

results_final.loc[results_final['Formulation Quality'] == 0, 'Solu'] = 0
results_final

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,...,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality,Solu,Solu_STD
0,112,0.573,0.427,0,0,0,0,0.000,0,0,...,2,0,Formulation 1,258.725650,4.254975,0.189512,0.037821,1,1146.0,0
1,113,0.558,0.442,0,0,0,0,0.000,0,0,...,3,0,Formulation 2,211.420437,3.978326,0.183822,0.023367,1,1116.0,0
2,114,0.596,0.404,0,0,0,0,0.000,0,0,...,1,0,Formulation 3,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
3,115,0.556,0.444,0,0,0,0,0.000,0,0,...,2,0,Formulation 4,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
4,116,0.596,0.404,0,0,0,0,0.000,0,0,...,1,0,Formulation 5,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
5,117,0.596,0.404,0,0,0,0,0.000,0,0,...,1,0,Formulation 6,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
6,118,0.596,0.404,0,0,0,0,0.000,0,0,...,1,0,Formulation 7,224.427667,3.430782,0.217428,0.097031,1,1192.0,0
7,119,0.552,0.448,0,0,0,0,0.000,0,0,...,2,0,Formulation 8,242.566169,11.567519,0.371350,0.000810,1,1104.0,0
8,120,0.596,0.404,0,0,0,0,0.000,0,0,...,1,0,Formulation 9,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
9,121,0.596,0.404,0,0,0,0,0.000,0,0,...,1,0,Formulation 10,1000.000000,0.000000,1.000000,0.000000,0,0.0,0


In [23]:
results_normalized = sdl.normalize(results_final)
results_normalized[['Formulation Quality','Size', 'Size_STD', 'Solu', 'Solu_STD', 'PDI', 'PDI_STD','Complexity']]

,Formulation Quality,Size,Size_STD,Solu,Solu_STD,PDI,PDI_STD,Complexity
0,1,0.258726,0.004255,0.573,0.0,0.189512,0.037821,0.166667
1,1,0.211420,0.003978,0.558,0.0,0.183822,0.023367,0.250000
2,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.083333
3,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.166667
4,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.083333
5,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.083333
6,1,0.224428,0.003431,0.596,0.0,0.217428,0.097031,0.083333
7,1,0.242566,0.011568,0.552,0.0,0.371350,0.000810,0.166667
8,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.083333
9,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.083333


In [24]:
results_normalized = sdl.normalize(results_final)
results_normalized[['Formulation Quality','Size', 'Size_STD', 'Solu', 'Solu_STD', 'PDI', 'PDI_STD','Complexity']]

,Formulation Quality,Size,Size_STD,Solu,Solu_STD,PDI,PDI_STD,Complexity
0,1,0.258726,0.004255,0.573,0.0,0.189512,0.037821,0.166667
1,1,0.211420,0.003978,0.558,0.0,0.183822,0.023367,0.250000
2,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.083333
3,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.166667
4,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.083333
5,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.083333
6,1,0.224428,0.003431,0.596,0.0,0.217428,0.097031,0.083333
7,1,0.242566,0.011568,0.552,0.0,0.371350,0.000810,0.166667
8,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.083333
9,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.083333


In [25]:
labeled_data_path = 'data/labeled_' + str(iteration) + '.xlsx'
labeled_norm_path = 'data/labeled_norm_' + str(iteration) + '.xlsx'


In [26]:
results_final.to_excel(labeled_data_path, index=False)
results_normalized.to_excel(labeled_norm_path, index=False)

# Load the labeled data to update the optimizer

In [27]:
labeled_norm = pd.read_excel(labeled_norm_path)
labeled_norm.head()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,...,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality,Solu,Solu_STD
0,112,0.573,0.427,0,0,0,0,0.0,0,0,...,0.166667,0,Formulation 1,0.258726,0.004255,0.189512,0.037821,1,0.573,0
1,113,0.558,0.442,0,0,0,0,0.0,0,0,...,0.250000,0,Formulation 2,0.211420,0.003978,0.183822,0.023367,1,0.558,0
2,114,0.596,0.404,0,0,0,0,0.0,0,0,...,0.083333,0,Formulation 3,1.000000,0.000000,1.000000,0.000000,0,0.000,0
3,115,0.556,0.444,0,0,0,0,0.0,0,0,...,0.166667,0,Formulation 4,1.000000,0.000000,1.000000,0.000000,0,0.000,0
4,116,0.596,0.404,0,0,0,0,0.0,0,0,...,0.083333,0,Formulation 5,1.000000,0.000000,1.000000,0.000000,0,0.000,0


In [28]:
ax_client_design = AxClient.load_from_json_file(ax_client_design_path)


In [29]:
loaded_optimizer_path = "optimizer/optimizer_load_" + str(iteration) + ".json"

In [30]:
loaded_ax_client = sdl.load_labeled_data(ax_client_design, labeled_norm_path)
loaded_ax_client.save_to_json_file(loaded_optimizer_path)

[INFO 08-07 08:34:25] ax.service.ax_client: Completed trial 112 with data: {'Solu': (0.573, 0), 'Size': (0.258726, 0.004255), 'PDI': (0.189512, 0.037821)}.
[INFO 08-07 08:34:25] ax.service.ax_client: Completed trial 113 with data: {'Solu': (0.558, 0), 'Size': (0.21142, 0.003978), 'PDI': (0.183822, 0.023367)}.
[INFO 08-07 08:34:25] ax.service.ax_client: Completed trial 114 with data: {'Solu': (0.0, 0), 'Size': (1.0, 0.0), 'PDI': (1.0, 0.0)}.
[INFO 08-07 08:34:25] ax.service.ax_client: Completed trial 115 with data: {'Solu': (0.0, 0), 'Size': (1.0, 0.0), 'PDI': (1.0, 0.0)}.
[INFO 08-07 08:34:25] ax.service.ax_client: Completed trial 116 with data: {'Solu': (0.0, 0), 'Size': (1.0, 0.0), 'PDI': (1.0, 0.0)}.
[INFO 08-07 08:34:25] ax.service.ax_client: Completed trial 117 with data: {'Solu': (0.0, 0), 'Size': (1.0, 0.0), 'PDI': (1.0, 0.0)}.
[INFO 08-07 08:34:25] ax.service.ax_client: Completed trial 118 with data: {'Solu': (0.596, 0), 'Size': (0.224428, 0.003431), 'PDI': (0.217428, 0.097031)